# 🕷️ Scrapy 사용법

Scrapy는 **웹 크롤링(Web Crawling)** 과 **웹 스크레이핑(Web Scraping)** 을 빠르고 효율적으로 할 수 있게 해주는 파이썬 프레임워크입니다.  


---

##  1. Scrapy 설치하기

### **1) pip로 설치**
Scrapy는 pip 한 줄로 설치할 수 있습니다.

```bash
pip install scrapy
```

##  2. Scrapy 프로젝트 만들기
```bash
scrapy startproject mycrawler


mycrawler/
 ├── mycrawler/
 │    ├── spiders/       ← 크롤러 파일 저장하는 곳
 │    ├── settings.py
 │    ├── items.py
 │    └── middlewares.py
 └── scrapy.cfg


예시 사이트: https://quotes.toscrape.com
 (연습용 크롤링 사이트)

아래 명령으로 템플릿 크롤러 생성:

cd mycrawler
scrapy genspider quotes quotes.toscrape.com

생성된 파일 경로:
mycrawler/spiders/quotes.py

프로젝트 폴더에서 실행:
scrapy crawl quotes

결과를 JSON 파일로 저장
scrapy crawl quotes -o quotes.json

{
  "text": "“The world as we have created it is a process of our thinking.”",
  "author": "Albert Einstein",
  "tags": ["change", "deep-thoughts", "thinking", "world"]
}

```



---

##  1. `spiders/` 폴더  
크롤러 파일을 저장하는 폴더입니다.  
우리가 직접 만드는 `quotes.py`, `news.py` 같은 스파이더들이 모두 여기에 들어갑니다.

- 한 스파이더가 한 사이트(혹은 특정 기능)를 담당
- 예:  

```bash
spiders/
├── quotes.py
├── products.py
└── blog.py
```

---

##  2. `settings.py`  
Scrapy 전체 동작 환경 설정 파일입니다.

여기서 다음 설정을 합니다:

| 설정 항목 | 설명 |
|----------|------|
| `USER_AGENT` | 크롤러의 HTTP User-Agent 변경 |
| `DOWNLOAD_DELAY` | 요청 간 딜레이 설정(방문 속도 조절) |
| `ROBOTSTXT_OBEY` | robots.txt 준수 여부 |
| `ITEM_PIPELINES` | 파이프라인 활성화 |
| `DOWNLOADER_MIDDLEWARES` | 다운로더 미들웨어 등록 |
| `CONCURRENT_REQUESTS` | 동시에 보낼 요청 수 제한 |

예시:

```python

# robots.txt 때문에 차단되는 경우(설정무시)
ROBOTSTXT_OBEY = False

# 어플리케이션 크롤링 차단의 경우
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0 Safari/537.36"

# 짧은 시간에 너무 많은 요청을 보내면 403/429 오류
DOWNLOAD_DELAY = 1   # 1초 딜레이

# 요청수 제한의 경우
CONCURRENT_REQUESTS = 5

# Header가 부족하거나 특정 Header를 요구하는 경우
# (일부 서버는 User-Agent 이외에도 Referer, Accept-Language 등을 확인
DEFAULT_REQUEST_HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept-Language": "ko-KR,ko;q=0.9",
    "Referer": "https://google.com"
}

# 과도한 크롤링으로 IP를 차단한 경우
DOWNLOADER_MIDDLEWARES = {
    'mycrawler.middlewares.ProxyMiddleware': 350,
}


```

## 3. items.py

크롤링한 데이터를 구조화하는 모델(데이터 스키마)을 정의하는 파일입니다.

예시:

```python
import scrapy

class QuoteItem(scrapy.Item):
    text = scrapy.Field()
    author = scrapy.Field()
    tags = scrapy.Field()
```

효과

추출 데이터의 구조를 명확히 하고

pipeline 등에서 데이터를 쉽게 처리 가능


## 4. middlewares.py

- 다운로더/스파이더 미들웨어를 설정하는 파일입니다.

미들웨어란?
Scrapy에서 요청(request) 과 응답(response) 사이를 가로채서 조작하는 중간 처리기입니다.

예시로 할 수 있는 작업:

- User-Agent 랜덤 변경
- Proxy 설정
- 요청/응답 로깅
- 쿠키 자동 처리

사용 예:
```yaml


DOWNLOADER_MIDDLEWARES = {
    'mycrawler.middlewares.MyCrawlerDownloaderMiddleware': 543,
}

과도한 크롤링으로 IP를 차단한 경우
class ProxyMiddleware:
    def process_request(self, request, spider):
        request.meta['proxy'] = "http://YOUR_PROXY_IP:PORT"

```

## 5. scrapy.cfg

프로젝트 최상단에 있으며 Scrapy 실행 설정을 관리합니다.
CLI에서 scrapy crawl 실행할 때 이 파일을 보고 어떤 프로젝트인지 인식합니다.
내용 예:
```yaml
[settings]
default = mycrawler.settings
```

| 파일/폴더              | 역할                          |
| ------------------ | --------------------------- |
| **spiders/**       | 크롤러(py 파일) 저장               |
| **settings.py**    | 프로젝트 설정(속도, 파이프라인, 미들웨어 등)  |
| **items.py**       | 크롤링 데이터 스키마 정의              |
| **middlewares.py** | 요청/응답을 중간에서 처리(UA, Proxy 등) |
| **scrapy.cfg**     | Scrapy 실행 설정(CLI에서 인식용)     |






| 문제 원인         | 해결 설정                            | 위치                           |
| ------------- | -------------------------------- | ---------------------------- |
| User-Agent 차단 | `USER_AGENT`                     | settings.py                  |
| 빠른 요청 차단      | `DOWNLOAD_DELAY`                 | settings.py                  |
| robots.txt 차단 | `ROBOTSTXT_OBEY = False`         | settings.py                  |
| Header 부족     | `DEFAULT_REQUEST_HEADERS`        | settings.py                  |
| IP 차단(Proxy)  | ProxyMiddleware                  | middlewares.py + settings.py |
| 로그인 필요        | `start_requests` + `FormRequest` | spider 파일                    |
| JS 렌더링 필요     | Playwright/Selenium              | settings.py + 설치             |



#### 로그인 필요한 사이트

로그인 없이 페이지를 막아두는 경우

✔ 해결 방법

```python
스파이더에서 start_requests() 재정의 + 쿠키 유지

def start_requests(self):
    login_url = "https://example.com/login"
    data = {"id": "user", "pw": "1234"}
    yield scrapy.FormRequest(url=login_url, formdata=data, callback=self.after_login)

def after_login(self, response):
    yield scrapy.Request("https://example.com/protected")
```
#### JavaScript 렌더링 때문에 빈 페이지만 보이는 경우

Scrapy는 기본적으로 JS 실행을 못함 → 데이터가 안 보임.

✔ 해결 방법
```python
Scrapy + Selenium

또는 ScrapingBee / Zyte API 사용

또는 scrapy-playwright 플러그인

scrapy-playwright 예시 (settings.py)
PLAYWRIGHT_BROWSER_TYPE = "chromium"
DOWNLOAD_HANDLERS = {
    "http": "scrapy_playwright.handler.ScrapyPlaywrightDownloadHandler",
    "https": "scrapy_playwright.handler.ScrapyPlaywrightDownloadHandler",
}


## quotes.py 예시


```python

import scrapy


class QuotesSpider(scrapy.Spider):
    name = "quotes"
    allowed_domains = ["quotes.toscrape.com"]
    start_urls = ["https://quotes.toscrape.com"]

    def parse(self, response):
        # 각 quote 블록 반복
        for quote in response.css("div.quote"):
            yield {
                "text": quote.css("span.text::text").get(),
                "author": quote.css("small.author::text").get(),
                "tags": quote.css("div.tags a.tag::text").getall(),
            }

        # 다음 페이지 있는 경우 계속 이동
        next_page = response.css("li.next a::attr(href)").get()
        if next_page:
            yield response.follow(next_page, callback=self.parse)


#  Proxy(프록시)의 역할

프록시는 **"내 대신 인터넷에 요청을 보내주는 중간 서버"**

---

#  1.  비유: “심부름꾼”

예를 들어 너가 어떤 가게에서 정보를 얻고 싶은데  
가게에서 너를 알아보고 “너는 안 돼!” 하고 막아버린다고 해보자.

그러면 **다른 사람(프록시)**에게 대신 가게에 가서 물어봐달라고 하는 거야.

- 가게 입장 → “어? 이 사람은 처음 보네? 들어오세요!”
- 사실은 → 그 사람이 네가 부탁한 *프록시*일 뿐

즉, 프록시는 **네 IP 주소를 숨기고 대신 요청을 보내주는 역할**을 해.

---

#  2. 기술적으로 말하면

프록시는:

- 네 컴퓨터(크롤러)와 서버(웹사이트) 사이에 있는 **중간 서버**
- 웹사이트는 프록시 IP를 진짜 접속자라고 생각

---

#  3. 프록시를 이용한 차단 회피

웹사이트는 크롤링이 너무 많으면  
> “이 IP 너무 많은 요청 보냈네? 차단!”

하고 내 IP를 막아버릴 수 있음.

하지만 프록시 IP를 사용하면:

- 내 IP는 숨겨지고
- 서버는 새로운 IP라고 생각해서 차단을 안 함
- 여러 개의 프록시를 돌리면 더 안전함

---

#  4. Scrapy에서 프록시를 쓰는 이유 정리

| 문제 | 프록시 사용 효과 |
|------|-------------------|
| 내 IP가 차단됨 | 다른 IP로 접속 가능 |
| 요청이 너무 많음 | 여러 프록시로 분산 가능 |
| 특정 국가에서만 접근 허용 | 해당 국가 IP 프록시 사용 |
| 웹사이트 유저 추적 회피 | IP 숨기기 가능 |

---



#  프록시(Proxy) 사이트 종류

프록시는 목적과 사용 방식에 따라 여러 종류로 나눌 수 있어요.  
크롤링에서 특히 많이 쓰이는 **프록시 서버 종류 + 추천 사이트**

---

#  1. **무료 프록시(Free Proxy)**  
아무나 무료로 사용하는 프록시 서버.

### ✔ 장점
- 무료

### ✔ 단점  
- 속도 느림  
- 금방 죽음(응답 없음)  
- 보안 취약  
- 신뢰성 낮음  
- 크롤링에 거의 부적합

### ✔ 무료 프록시 사이트 예  
- FreeProxyList (https://free-proxy-list.net)  
- ProxyScrape (https://proxyscrape.com/free-proxy-list)  
- Spys.one  
- Geonode free list  


---

#  2. **프라이빗 프록시(Private Proxy)**  
유료로 구매하는 개인 전용 IP  
(너만 사용 → 빠르고 안정적)

### ✔ 장점
- 속도 매우 빠름  
- 거의 차단 안 됨  
- 안정적  
- 크롤링 실무에 적합  

### ✔ 단점  
- 유료(보통 월 단위)

### ✔ 추천 사이트  
- Bright Data (전 세계 제일 큰 프록시 업체)  
- ScraperAPI  
- Oxylabs  
- Smartproxy  
- Webshare (가성비 좋음)

---

#  3. **회전 프록시(Rotating Proxy)**  
요청할 때마다 자동으로 프록시 IP가 바뀜  
→ 크롤링에 가장 많이 사용됨

### ✔ 장점
- IP 자동 로테이션  
- 차단 위험 낮음  
- 장시간 크롤링에 최적

### ✔ 단점
- 유료

### ✔ 추천 사이트  
- Bright Data Rotating Proxy  
- Oxylabs Rotating Residential  
- ScrapingBee (API 방식)  
- Zyte Smart Proxy Manager (Scrapy 기업)

---

#  4. **레지덴셜 프록시(Residential Proxy)**  
"진짜 가정집에서 쓰는 IP"를 제공  
(크롤링 탐지 우회에 최강)

### ✔ 장점
- 웹사이트에서 실제 사람으로 인식  
- 차단 거의 없음

### ✔ 단점
- 비쌈  
- 대량 크롤링은 비용 부담

### ✔ 추천 사이트  
- Bright Data Residential  
- Smartproxy Residential  
- Oxylabs Residential

---

#  5. **데이터센터 프록시(Datacenter Proxy)**  
데이터센터에서 제공하는 IP  
(지속적으로 빠름)

### ✔ 장점
- 빠름  
- 싸고 대량으로 사용 가능  

### ✔ 단점
- 레지덴셜보다 차단 확률 높음

### ✔ 추천 사이트  
- Webshare  
- Smartproxy  
- IPRoyal  
- ProxyRack

---

#  종류별 핵심 요약

| 종류 | 특징 | 크롤링 적합도 |
|------|------|---------------|
| 무료 프록시 | 불안정, 느림 | ★☆☆☆☆ (연습용) |
| 프라이빗 프록시 | 개인 전용, 빠름 | ★★★★☆ |
| 회전 프록시 | 요청마다 IP 변경 | ★★★★★ (최적) |
| 레지덴셜 프록시 | 가정집 IP, 탐지 우회 최강 | ★★★★★ |
| 데이터센터 프록시 | 빠르고 저렴 | ★★★★☆ |

---

#  크롤링 용으로는 어떤 프록시가 최고인가?  
 **회전 프록시 + 레지덴셜 IP 조합**  
(가장 자연스럽고 차단되지 않음)

---

